In [1]:
import polars as pl # similar pandas
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

# 1. LOAD DATA
## Load CSV/Parquet files into Polars DataFrames
#### If Parquet exists, read it directly; otherwise read CSV, write Parquet, then read

In [1]:
# CSV and Parquet directories
csv_dir = Path("../data/csv")
parquet_dir = Path("../data/parquet")
parquet_dir.mkdir(exist_ok=True)  # create folder if not exists

# List of tables
tables = [
    "distribution_centers",
    "events",
    "inventory_items",
    "orders",
    "order_items",
    "products",
    "users"]

# Dictionary to store DataFrames
dfs = {}

for table in tables:
    csv_path = csv_dir / f"{table}.csv"
    parquet_path = parquet_dir / f"{table}.parquet"    
    try:
        if parquet_path.exists():
            print(f"{parquet_path} already exists, reading Parquet...")
        else:
            print(f"{parquet_path} not found, reading CSV and writing Parquet...")
            df_csv = pl.read_csv(csv_path)
            df_csv.write_parquet(parquet_path)
            print(f"{parquet_path} has been created.\n")
        
        # Read the Parquet file
        dfs[table] = pl.read_parquet(parquet_path)
        print(f"{table}: {dfs[table].shape[0]} rows, {dfs[table].shape[1]} columns\n")
    
    except Exception as e:
        print(f"Error reading {table}: {e}\n")

NameError: name 'Path' is not defined

In [3]:
# Get references from dfs to separate variables
# dfs still keeps initial values
df_dc = dfs["distribution_centers"]
df_events = dfs["events"]
df_inv_items = dfs["inventory_items"]
df_orders = dfs["orders"]
df_order_items = dfs["order_items"]
df_products = dfs["products"]
df_users = dfs["users"]

# 2. DATA CLEANING & TYPE CONVERSION

## 2.1 Check for missing values

In [4]:
def check_na_pct(dfs:dict):
    for name, df in dfs.items():
        print(f"=== {name} ===")
        na_info = {}
        for c in df.columns:
            na_count = df[c].null_count()
            na_pct = (na_count / df.height) * 100
            if na_count > 0:
                na_info[c] = (na_count, na_pct)
        if na_info:
            for na_col, (na_count, na_pct) in na_info.items():
                print(f"{na_col}: {na_count} NA ({na_pct:.2f}%)")
        else:
            print("No missing values")
        print()

check_na_pct(dfs)

=== distribution_centers ===
No missing values

=== events ===
user_id: 1125671 NA (46.29%)

=== inventory_items ===
sold_at: 308946 NA (62.96%)
product_name: 29 NA (0.01%)
product_brand: 401 NA (0.08%)

=== orders ===
returned_at: 112696 NA (89.99%)
shipped_at: 43765 NA (34.95%)
delivered_at: 81342 NA (64.96%)

=== order_items ===
shipped_at: 63478 NA (34.92%)
delivered_at: 117918 NA (64.88%)
returned_at: 163527 NA (89.97%)

=== products ===
name: 2 NA (0.01%)
brand: 24 NA (0.08%)

=== users ===
No missing values



## Fill missing values

In [5]:
# === events ===
df_events = df_events.with_columns(
    pl.col("user_id").fill_null("anonymous")
)

# === inventory_items ===
df_inv_items = df_inv_items.with_columns([
    pl.col("sold_at").is_not_null().alias("is_sold"),
    pl.col("product_name").fill_null("unknown"),
    pl.col("product_brand").fill_null("unknown")
])
# === products ===
df_products = df_products.with_columns([
    pl.col("name").fill_null("unknown"),
    pl.col("brand").fill_null("unknown")
])

# === Update dictionary ===
# Assign new data to dfs after processing
# polar does not update as pandas
dfs["inventory_items"] = df_inv_items
dfs["events"] = df_events
dfs["products"] = df_products

check_na_pct(dfs)

=== distribution_centers ===
No missing values

=== events ===
No missing values

=== inventory_items ===
sold_at: 308946 NA (62.96%)

=== orders ===
returned_at: 112696 NA (89.99%)
shipped_at: 43765 NA (34.95%)
delivered_at: 81342 NA (64.96%)

=== order_items ===
shipped_at: 63478 NA (34.92%)
delivered_at: 117918 NA (64.88%)
returned_at: 163527 NA (89.97%)

=== products ===
No missing values

=== users ===
No missing values



# 3. EDA for each table separately

## 3.1 Data Summary

In [6]:
# ===== DISTRIBUTION CENTERS =====#
dc_total = len(df_dc)

# ===== PRODUCTS =====#
pro_len = len(df_products)
pro_cat = df_products["category"].n_unique()
pro_brand = df_products["brand"].n_unique()
pro_dpm = df_products["department"].n_unique()
pro_dc = df_products["distribution_center_id"].n_unique()

# ===== USERS =====#
user_len = len(df_users)

user_age_min = df_users["age"].min()
user_age_max = df_users["age"].max()

user_age = (
    df_users.group_by("age").agg(pl.len().alias("Total")).sort("age")
)

user_gender = df_users["gender"].n_unique()
user_state = df_users["state"].n_unique()
user_city = df_users["city"].n_unique()
user_country = df_users["country"].n_unique()
user_traffic = df_users["traffic_source"].n_unique()

# ===== ORDERS =====#
order_len = len(df_orders)


# ===== PRINT =====#
print(f"#------ DATASET SUMMARY ------#\n")
print(
    f"#----- DISTRIBUTION CENTERS -----#"
    f"\n- {dc_total} distribution centers.\n"
    f"\n#----- PRODUCTS -----#"
    f"\n- {pro_len} products, across {pro_cat} categories, {pro_brand} brands,"
    f" and {pro_dpm} departments, "
    f"distributed across {pro_dc} distribution centers.\n"
    f"\n#----- USERS -----#"
    f"\n- {user_len} users, ages {user_age_min}-{user_age_max},"
    f"\n- Users span {user_gender} genders and are globally distributed across "
    f"{user_city} cities, {user_state} states, and {user_country} countries."
    f"\n- The data captures {user_traffic} distinct traffic sources.\n"
    f"\n#----- ORDERS -----#"
    f"\n- {order_len} orders"
)

#------ DATASET SUMMARY ------#

#----- DISTRIBUTION CENTERS -----#
- 10 distribution centers.

#----- PRODUCTS -----#
- 29120 products, across 26 categories, 2757 brands, and 2 departments, distributed across 10 distribution centers.

#----- USERS -----#
- 100000 users, ages 12-70,
- Users span 2 genders and are globally distributed across 7884 cities, 229 states, and 16 countries.
- The data captures 5 distinct traffic sources.

#----- ORDERS -----#
- 125226 orders


## 3.2 distribution_centers.csv

In [7]:
# get central point of map (mean lat/lon)
dc_center_lat = df_dc["latitude"].mean()
dc_center_lon = df_dc["longitude"].mean()

dc_map = folium.Map(
    location=[dc_center_lat, dc_center_lon],
    zoom_start = 5 # number of points
)

dc_cluster = MarkerCluster().add_to(dc_map)

for row in df_dc.iter_rows(named=True):
    folium.Marker(location=[row["latitude"], 
                            row["longitude"]]).add_to(dc_cluster)
dc_map

## 3.3 products.csv

In [8]:
pro_cat_total = (
    df_products.group_by("category").agg(pl.len().alias("Total")).sort("category")
)

pro_brand_total = (
    df_products.group_by("brand").agg(pl.len().alias("Total")).sort("brand")
)

pro_dpm_total = (
    df_products.group_by("department").agg(pl.len().alias("Total"))
)

pro_dc_total = df_products.group_by("distribution_center_id").agg([
    pl.col("id").n_unique().alias("n_products"),
    pl.col("category").n_unique().alias("n_cats"),
    pl.col("brand").n_unique().alias("n_brands")
]).sort("distribution_center_id", descending = False)

print(f"\n#======== CATEGORY ========#")
for row in pro_cat_total.iter_rows():
    cat, total = row
    print(f"- {cat}: {total} items")
    
print(f"\n#======== BRAND ========#")
for row in pro_brand_total.iter_rows():
    brand, total = row
    print(f"- {brand}: {total} items")

print(f"#======== DEPARTMENT ========#")
for row in pro_dpm_total.iter_rows():
    dpm, total = row
    print(f"- {dpm}: {total} items")

print(f"\n#======== DISTRIBUTION CENTERS ========#")
for row in pro_dc_total.iter_rows():
    dc, pros, cats, brands = row
    print(f"DC {dc}: {pros} products across {cats} categories and {brands} brands")


#======== CATEGORY ========#
- Accessories: 1559 items
- Active: 1432 items
- Blazers & Jackets: 561 items
- Clothing Sets: 37 items
- Dresses: 955 items
- Fashion Hoodies & Sweatshirts: 1866 items
- Intimates: 2363 items
- Jeans: 1999 items
- Jumpsuits & Rompers: 162 items
- Leggings: 564 items
- Maternity: 898 items
- Outerwear & Coats: 1420 items
- Pants: 1041 items
- Pants & Capris: 613 items
- Plus: 758 items
- Shorts: 1765 items
- Skirts: 367 items
- Sleep & Lounge: 1771 items
- Socks: 905 items
- Socks & Hosiery: 666 items
- Suits: 188 items
- Suits & Sport Coats: 739 items
- Sweaters: 1737 items
- Swim: 1798 items
- Tops & Tees: 1868 items
- Underwear: 1088 items

#======== BRAND ========#
- !it Jeans: 13 items
- '47 Brand: 1 items
- 007Lingerie: 3 items
- 10 Deep: 1 items
- 106Shades: 6 items
- 12XLShop Inc.: 1 items
- 180s: 22 items
- 1826: 1 items
- 1WorldSarong: 3 items
- 1veMoon: 1 items
- 1vemoon: 3 items
- 2(x)ist: 56 items
- 2117 of Sweden: 1 items
- 2EROS: 1 items
- 2

## 3.4 users.csv

In [9]:
user_age_total = df_users.group_by("age").agg(pl.len().alias("Total")).sort("age")
user_gender_total = df_users.group_by("gender").agg(pl.len().alias("Total"))
user_state_total = df_users.group_by("state").agg(pl.len().alias("Total"))
user_city_total = df_users.group_by("city").agg(pl.len().alias("Total"))
user_country_total = df_users.group_by("country").agg(pl.len().alias("Total"))
user_traffic_total = df_users.group_by("traffic_source").agg(pl.len().alias("Total"))

print(f"\n#======== AGE ========#")
for row in user_age_total.iter_rows():
    age, total = row
    print(f"- {age}: {total} people")

print(f"\n#======== GENDER ========#")
for row in user_gender_total.iter_rows():
    gender, total = row
    print(f"- {gender}: {total} people")

print(f"\n#======== STATE ========#")
for row in user_state_total.iter_rows():
    state, total = row
    print(f"- {state}: {total} people")

print(f"\n#======== CITY ========#")
for row in user_city_total.iter_rows():
    city, total = row
    print(f"- {city}: {total} people")

print(f"\n#======== COUNTRY ========#")
for row in user_country_total.iter_rows():
    country, total = row
    print(f"- {country}: {total} people")

print(f"\n#======== TRAFFIC SOURCE ========#")
for row in user_traffic_total.iter_rows():
    traffic, total = row
    print(f"- {traffic}: {total} people")


#======== AGE ========#
- 12: 1685 people
- 13: 1642 people
- 14: 1702 people
- 15: 1673 people
- 16: 1681 people
- 17: 1767 people
- 18: 1751 people
- 19: 1616 people
- 20: 1770 people
- 21: 1754 people
- 22: 1657 people
- 23: 1647 people
- 24: 1701 people
- 25: 1693 people
- 26: 1639 people
- 27: 1632 people
- 28: 1780 people
- 29: 1625 people
- 30: 1687 people
- 31: 1624 people
- 32: 1663 people
- 33: 1802 people
- 34: 1658 people
- 35: 1724 people
- 36: 1687 people
- 37: 1697 people
- 38: 1662 people
- 39: 1656 people
- 40: 1695 people
- 41: 1757 people
- 42: 1687 people
- 43: 1644 people
- 44: 1730 people
- 45: 1706 people
- 46: 1674 people
- 47: 1655 people
- 48: 1698 people
- 49: 1642 people
- 50: 1727 people
- 51: 1722 people
- 52: 1675 people
- 53: 1715 people
- 54: 1668 people
- 55: 1797 people
- 56: 1736 people
- 57: 1656 people
- 58: 1730 people
- 59: 1742 people
- 60: 1679 people
- 61: 1631 people
- 62: 1682 people
- 63: 1716 people
- 64: 1706 people
- 65: 1724 people
- 6

In [10]:
# Center map theo users
center_lat = df_users["latitude"].mean()
center_lon = df_users["longitude"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=2,
    tiles="CartoDB positron"
)

# Users heatmap (NHANH)
user_points = df_users.select(["latitude", "longitude"]).to_numpy().tolist()
HeatMap(user_points, radius=10).add_to(m)

# Distribution centers (RÕ)
for row in df_dc.iter_rows(named=True):
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="red", icon="home"),
        popup=f"DC {row['id']}"
    ).add_to(m)
m

## 3.5 orders.csv

In [11]:
order_stt_total = df_orders.group_by("status").agg(pl.len().alias("Total"))
order_item_total = df_orders.group_by("num_of_item").agg(
    pl.len().alias("Total")).sort("num_of_item")

print(f"\n#======== STATUS ========#")
for row in order_stt_total.iter_rows():
    stt, total = row
    print(f"- {stt}: {total} orders")

print(f"\n#======== NUMBER OF ITEMS PER ORDERS ========#")
for row in order_item_total.iter_rows():
    n_item, total = row
    print(f"- {n_item} items: {total} orders")


#======== STATUS ========#
- Complete: 31354 orders
- Cancelled: 18609 orders
- Returned: 12530 orders
- Processing: 25156 orders
- Shipped: 37577 orders

#======== NUMBER OF ITEMS PER ORDERS ========#
- 1 items: 87712 orders
- 2 items: 24867 orders
- 3 items: 6275 orders
- 4 items: 6372 orders


In [12]:
order_item_stt = (
    df_orders.
        group_by(["status", "num_of_item"]).
        agg(pl.len().alias("Total")).
        sort(["status", "num_of_item"])
)
order_item_stt

status,num_of_item,Total
str,i64,u32
"""Cancelled""",1,13008
"""Cancelled""",2,3722
"""Cancelled""",3,878
"""Cancelled""",4,1001
"""Complete""",1,21915
…,…,…
"""Returned""",4,632
"""Shipped""",1,26381
"""Shipped""",2,7439


### Convert to datetime

In [13]:
def parse_datetime_cols(df, cols, tz="UTC"):
    exprs = []
    for c in cols:
        if df[c].dtype == pl.Utf8:
            exprs.append(pl.col(c).str.to_datetime(time_zone=tz).alias(c))
        else:
            exprs.append(pl.col(c))
    return df.with_columns(exprs)

In [14]:
df_orders = parse_datetime_cols(
    df_orders,
    ["created_at", "shipped_at", "delivered_at", "returned_at"]
)

### 3.5.1 Stage Lead Time

In [15]:
# dt.total_hours()
# dt.total_days()
df_orders = df_orders.with_columns([
    (pl.col("shipped_at") - pl.col("created_at"))
      .dt.total_hours().alias("created_shipped_hrs"),
    
    (pl.col("delivered_at") - pl.col("shipped_at"))
      .dt.total_hours().alias("shipped_delivered_hrs"),

    (pl.col("returned_at") - pl.col("delivered_at"))
      .dt.total_hours().alias("delivered_returned_hrs"),
])

avg_cs_hrs = df_orders.select(
    pl.col("created_shipped_hrs").mean().round(2)).item()

avg_sd_hrs = df_orders.select(
    pl.col("shipped_delivered_hrs").mean().round(2)).item()

avg_dr_hrs = df_orders.select(
    pl.col("delivered_returned_hrs").mean().round(2)).item()

print(
    f"AVERAGE STAGE LEAD TIME"
    f"\n- created → shipped: {avg_cs_hrs} hrs (~{avg_cs_hrs/24:.1f} days)"
    f"\n- shipped → delivered: {avg_sd_hrs} hrs (~{avg_sd_hrs/24:.1f} days)"
    f"\n- delivered → returned: {avg_dr_hrs} hrs (~{avg_dr_hrs/24:.1f} days)"
)

AVERAGE STAGE LEAD TIME
- created → shipped: 35.44 hrs (~1.5 days)
- shipped → delivered: 59.75 hrs (~2.5 days)
- delivered → returned: 35.58 hrs (~1.5 days)


### 3.5.2 End-to-end lead time

In [24]:
# shipped --> created → shipped
df_shipped = df_orders.filter(pl.col("status") == "Shipped")
df_shipped = df_shipped.with_columns([
    (pl.col("shipped_at") - pl.col("created_at"))
      .dt.total_hours().alias("shipped_hrs")
])

# completed --> created → shipped → delivered
df_complete = df_orders.filter(pl.col("status") == "Complete")
df_complete = df_complete.with_columns([
    (pl.col("delivered_at") - pl.col("created_at"))
      .dt.total_hours().alias("complete_hrs")
])

# returned --> created → shipped → delivered → returned
df_returned = df_orders.filter(pl.col("status") == "Returned")
df_returned = df_returned.with_columns([
    (pl.col("returned_at") - pl.col("created_at"))
      .dt.total_hours().alias("returned_hrs")
])

avg_shipped_hrs = df_shipped.select(
    pl.col("shipped_hrs").mean().round(2)).item()

avg_complete_hrs = df_complete.select(
    pl.col("complete_hrs").mean().round(2)).item()

avg_returned_hrs = df_returned.select(
    pl.col("returned_hrs").mean().round(2)).item()

print(
    f"END-TO-END LEAD TIME"
    f"\n- Shipped: {avg_shipped_hrs} hrs (~{avg_shipped_hrs/24:.1f} days)"
    f"\n- Complete: {avg_complete_hrs} hrs (~{avg_complete_hrs/24:.1f} days)"
    f"\n- Returned: {avg_returned_hrs} hrs (~{avg_returned_hrs/24:.1f} days)"
)

END-TO-END LEAD TIME
- Shipped: 35.39 hrs (~1.5 days)
- Complete: 95.9 hrs (~4.0 days)
- Returned: 131.37 hrs (~5.5 days)


In [26]:
df_e2e = df_orders.with_columns(
    pl.when(pl.col("status") == "Shipped")
      .then((pl.col("shipped_at") - pl.col("created_at")).dt.total_hours())
      .when(pl.col("status") == "Complete")
      .then((pl.col("delivered_at") - pl.col("created_at")).dt.total_hours())
      .when(pl.col("status") == "Returned")
      .then((pl.col("returned_at") - pl.col("created_at")).dt.total_hours())
      .otherwise(None)
      .alias("end_to_end_hrs")
)
df_e2e.group_by("status").agg(
    pl.mean("end_to_end_hrs").round(2)
)


status,end_to_end_hrs
str,f64
"""Complete""",95.9
"""Returned""",131.37
"""Processing""",null
"""Cancelled""",null
"""Shipped""",35.39


## events.csv

In [27]:
df_events.head()

id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
i64,str,i64,str,str,str,str,str,str,str,str,str,str
2198523,"""anonymous""",3,"""83889ed2-2adc-4b9a-af5d-154f69…","""2021-06-17 17:30:00+00:00""","""138.143.9.202""","""São Paulo""","""São Paulo""","""02675-031""","""Chrome""","""Adwords""","""/cancel""","""cancel"""
1773216,"""anonymous""",3,"""7a3fc3f2-e84f-44fe-8876-eff767…","""2020-08-07 08:41:00+00:00""","""85.114.141.79""","""Santa Isabel""","""São Paulo""","""07500-000""","""Safari""","""Adwords""","""/cancel""","""cancel"""
2380515,"""anonymous""",3,"""13d9b2fb-eee1-43fd-965c-267b38…","""2021-02-15 18:48:00+00:00""","""169.250.255.132""","""Mairiporã""","""São Paulo""","""07600-000""","""IE""","""Adwords""","""/cancel""","""cancel"""
2250597,"""anonymous""",3,"""96f1d44e-9621-463c-954c-d8deb7…","""2022-03-30 10:56:00+00:00""","""137.25.222.160""","""Cajamar""","""São Paulo""","""07750-000""","""Chrome""","""Adwords""","""/cancel""","""cancel"""
1834446,"""anonymous""",3,"""d09dce10-a7cb-47d3-a9af-449755…","""2019-09-05 01:18:00+00:00""","""161.114.4.174""","""São Paulo""","""São Paulo""","""09581-680""","""Chrome""","""Email""","""/cancel""","""cancel"""
